**CCDDNET TRAINING CODE**

In [7]:
import os
import numpy as np
import tensorflow as tf
import cv2

# Paths to the noisy and clean image folders for training
train_noisy_folder = '\noisy\\noisy_10\\R\\bit_plane_0'
train_clean_folder = '\clean\\R\\bit_plane_0'


#from google.colab.patches import cv2_imshow

def load_images_from_folder(folder):
    images = []
    for filename in os.listdir(folder):
        img = cv2.imread(os.path.join(folder, filename), cv2.IMREAD_GRAYSCALE)

        if img is not None:
            img = img.astype(np.float32) / 255.0  # Normalize to [0, 1]
            img[img > 0.5] = 1  # Convert to binary (1 if greater than 0.5)
            img[img <= 0.5] = 0  # Convert to binary (0 if less than or equal to 0.5)
            images.append(img)
    return np.array(images)

def extract_patches(images, patch_size, stride):
    patches = []
    for img in images:
        img = img.reshape(img.shape[0], img.shape[1], 1)  # Add channel dimension
        for i in range(0, img.shape[0] - patch_size + 1, stride):
            for j in range(0, img.shape[1] - patch_size + 1, stride):
                patch = img[i:i+patch_size, j:j+patch_size]
                patches.append(patch)
    return np.array(patches)

# Load training images
X_train_images = load_images_from_folder(train_noisy_folder)
Y_train_images = load_images_from_folder(train_clean_folder)

# Extract patches
patch_size = 40
stride = 10
X_train = extract_patches(X_train_images, patch_size, stride)
Y_train = extract_patches(Y_train_images, patch_size, stride)

# Define ECNDNet model for binary images
def ECNDNet(input_shape):
    inputs = tf.keras.layers.Input(shape=input_shape)

    def conv_block(x, filters, kernel_size=3, dilation_rate=1, use_bn=True, use_relu=True):
        x = tf.keras.layers.Conv2D(filters, kernel_size, padding='same', dilation_rate=dilation_rate)(x)
        if use_bn:
            x = tf.keras.layers.BatchNormalization()(x)
        if use_relu:
            x = tf.keras.layers.Activation('relu')(x)
        return x

    x = conv_block(inputs, 64, use_bn=False)

    # Adding residual connections after every two convolution blocks
    for i in range(1, 16):
        x = conv_block(x, 64)
        if i % 2 == 0:  # After every 2 convolution blocks
            # Ensure that the input to the residual connection is the output of the conv_block
            residual = tf.keras.layers.Conv2D(64, 1, padding='same')(x)
            x = tf.keras.layers.Add()([x, residual])  # Residual connection

        # Add dilation rate on specific layers
        if i in [2, 5, 9, 12]:
            x = conv_block(x, 64, dilation_rate=2)

    x = conv_block(x, 64, use_bn=False)
    outputs = tf.keras.layers.Conv2D(1, 1, padding='same', activation='sigmoid')(x)  # Single channel output

    return tf.keras.models.Model(inputs, outputs)

model = ECNDNet((patch_size, patch_size, 1))  # Ensure input shape is (40, 40, 1)

# Compile the model
initial_lr = 1e-3
model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=initial_lr, beta_1=0.9, beta_2=0.999, epsilon=1e-8),
              loss=tf.keras.losses.BinaryCrossentropy())

# Learning rate schedule
lr_schedule = tf.keras.callbacks.LearningRateScheduler(lambda epoch: initial_lr * (1e-3 ** (epoch / 180)))

# Custom callback to print training loss after each epoch
class PrintTrainingLoss(tf.keras.callbacks.Callback):
    def on_epoch_end(self, epoch, logs=None):
        print(f'Epoch {epoch + 1}: Training Loss: {logs["loss"]:.4f}')

# Create learning rate scheduler callback
checkpoint_dir = "\checkpoint"
checkpoint_path = checkpoint_dir + 'bit_denoising_model_{epoch:02d}.pb'
checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(filepath=checkpoint_path, save_weights_only=False, save_best_only=False, monitor='val_loss', mode='min', verbose=1, save_freq='epoch', period=5)

log_path = '\log.csv'
csv_logger = tf.keras.callbacks.CSVLogger(log_path)

# Train the model
model.fit(X_train, Y_train, epochs=50, batch_size=32, callbacks=[lr_schedule, checkpoint_callback, csv_logger, PrintTrainingLoss()], validation_split=0.1)

# Save the model
# BPS_DATASET is the folder where the model is saved
model_path = "\BPS_DATASET"
model.save(model_path)

# Function to reassemble patches into the original image size
def reassemble_patches(patches, image_shape, patch_size, stride):
    h, w = image_shape[:2]
    reassembled_image = np.zeros((h, w))
    patch_counts = np.zeros((h, w))

    patch_idx = 0
    for i in range(0, h - patch_size + 1, stride):
        for j in range(0, w - patch_size + 1, stride):
            reassembled_image[i:i+patch_size, j:j+patch_size] += patches[patch_idx]
            patch_counts[i:i+patch_size, j:j+patch_size] += 1
            patch_idx += 1

    reassembled_image /= patch_counts
    return np.clip(reassembled_image, 0, 1)  # Ensure values are in [0, 1]

# Denoise the test images using the trained model (add your test code here)


Epoch 1/50
327/327 [==============================] - 267s 762ms/step - loss: 0.6848 - val_loss: 0.6915 - lr: 0.0010
Epoch 2/50
327/327 [==============================] - 246s 753ms/step - loss: 0.6784 - val_loss: 0.6914 - lr: 9.6235e-04
Epoch 3/50
327/327 [==============================] - 249s 761ms/step - loss: 0.6760 - val_loss: 0.6930 - lr: 9.2612e-04
Epoch 4/50
327/327 [==============================] - 251s 766ms/step - loss: 0.6741 - val_loss: 0.9299 - lr: 8.9125e-04
Epoch 5/50
327/327 [==============================] - ETA: 0s - loss: 0.6732
Epoch 5: saving model to D:\PHD_WORK\3rd_experiment\BPS_DATASET\checkpointbit_denoising_model_05.pb


INFO:tensorflow:Assets written to: D:\PHD_WORK\3rd_experiment\BPS_DATASET\checkpointbit_denoising_model_05.pb\assets


INFO:tensorflow:Assets written to: D:\PHD_WORK\3rd_experiment\BPS_DATASET\checkpointbit_denoising_model_05.pb\assets


Epoch 5: Training Loss: 0.6732
327/327 [==============================] - 259s 792ms/step - loss: 0.6732 - val_loss: 0.6923 - lr: 8.5770e-04
Epoch 6/50
327/327 [==============================] - 245s 749ms/step - loss: 0.6737 - val_loss: 0.7259 - lr: 8.2540e-04
Epoch 7/50
327/327 [==============================] - 251s 769ms/step - loss: 0.6732 - val_loss: 0.7063 - lr: 7.9433e-04
Epoch 8/50
327/327 [==============================] - 253s 774ms/step - loss: 0.6724 - val_loss: 0.7158 - lr: 7.6442e-04
Epoch 9/50
327/327 [==============================] - 253s 773ms/step - loss: 0.6718 - val_loss: 0.7002 - lr: 7.3564e-04
Epoch 10/50
327/327 [==============================] - ETA: 0s - loss: 0.6714
Epoch 10: saving model to D:\PHD_WORK\3rd_experiment\BPS_DATASET\checkpointbit_denoising_model_10.pb


INFO:tensorflow:Assets written to: D:\PHD_WORK\3rd_experiment\BPS_DATASET\checkpointbit_denoising_model_10.pb\assets


...

INFO:tensorflow:Assets written to: D:\PHD_WORK\3rd_experiment\BPS_DATASET\checkpointbit_denoising_model_50.pb\assets


INFO:tensorflow:Assets written to: D:\PHD_WORK\3rd_experiment\BPS_DATASET\checkpointbit_denoising_model_50.pb\assets


Epoch 50: Training Loss: 0.3718
327/327 [==============================] - 251s 766ms/step - loss: 0.3718 - val_loss: 1.2009 - lr: 1.5252e-04


INFO:tensorflow:Assets written to: D:\PHD_WORK\3rd_experiment\BPS_DATASET\assets


INFO:tensorflow:Assets written to: D:\PHD_WORK\3rd_experiment\BPS_DATASET\assets


# **CONTINUE FOR FURTHER EPOCHS ON DIFFERENT DATASET**

In [1]:
import os
import numpy as np
import tensorflow as tf
import cv2

import time

# # track start time of program
start = time.time()

# Restrict TensorFlow to only allocate 2GB of GPU memory
gpus = tf.config.experimental.list_physical_devices('GPU')
if gpus:
    try:
        tf.config.experimental.set_virtual_device_configuration(
            gpus[0],
            [tf.config.experimental.VirtualDeviceConfiguration(memory_limit=2048)])  # 2GB limit
    except RuntimeError as e:
        print(e)


# Function to load images from a folder
def load_images_from_folder(folder):
    images = []
    for filename in os.listdir(folder):
        img = cv2.imread(os.path.join(folder, filename), cv2.IMREAD_GRAYSCALE)
        if img is not None:
            img = img.astype(np.float32) / 255.0  # Normalize to [0, 1]
            img[img > 0.5] = 1  # Convert to binary
            img[img <= 0.5] = 0
            images.append(img)
    return np.array(images)

# Function to extract patches
def extract_patches(images, patch_size, stride):
    patches = []
    for img in images:
        img = img.reshape(img.shape[0], img.shape[1], 1)  # Add channel dimension
        for i in range(0, img.shape[0] - patch_size + 1, stride):
            for j in range(0, img.shape[1] - patch_size + 1, stride):
                patch = img[i:i+patch_size, j:j+patch_size]
                patches.append(patch)
    return np.array(patches)

# Paths to the new noisy and clean image folders for training
new_train_noisy_folder = 'D:\\PHD_WORK\\3rd_experiment\\BPS_DATASET\\noisy\\noisy_10\\G\\bit_plane_3'
new_train_clean_folder = 'D:\\PHD_WORK\\3rd_experiment\\BPS_DATASET\\clean\\G\\bit_plane_3'

# Load new training images
X_new_train_images = load_images_from_folder(new_train_noisy_folder)
Y_new_train_images = load_images_from_folder(new_train_clean_folder)

# Extract patches from the new dataset
patch_size = 40
stride = 10
X_new_train = extract_patches(X_new_train_images, patch_size, stride)
Y_new_train = extract_patches(Y_new_train_images, patch_size, stride)

# Load the previously trained model
model_path = "D:\\PHD_WORK\\3rd_experiment\\BPS_DATASET\\noisy_10_model\\noisy_10_B_bitplane_model\\checkpoint_b_10_2\\bit_denoising_model_50.pb"
model = tf.keras.models.load_model(model_path)

# Compile the model (if not already compiled)
initial_lr = 1e-3
model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=initial_lr),
              loss=tf.keras.losses.BinaryCrossentropy())

# Callbacks for training
checkpoint_dir = "D:\\PHD_WORK\\3rd_experiment\\BPS_DATASET\\noisy_10_model\\noisy_10_G_bitplane_model\\checkpoint_g_10_3"
checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(
    filepath=os.path.join(checkpoint_dir, 'bit_denoising_model_{epoch:02d}.pb'),
    save_weights_only=False, save_best_only=False, mode='min', verbose=1, save_freq='epoch', period=5
)

log_path = 'D:\\PHD_WORK\\3rd_experiment\\BPS_DATASET\\noisy_10_model\\noisy_10_G_bitplane_model\\checkpoint_g_10_3\\log1.csv'
csv_logger = tf.keras.callbacks.CSVLogger(log_path)

class PrintTrainingLoss(tf.keras.callbacks.Callback):
    def on_epoch_end(self, epoch, logs=None):
        print(f'Epoch {epoch + 1}: Training Loss: {logs["loss"]:.4f}')

# Train the model on the new dataset for additional epochs
additional_epochs = 20  # Specify the number of additional epochs
model.fit(X_new_train, Y_new_train, epochs=50, batch_size=32, callbacks=[checkpoint_callback, csv_logger, PrintTrainingLoss()])

# Save the model after additional training
new_model_path = "D:\\PHD_WORK\\3rd_experiment\\BPS_DATASET\\bit_denoising_model_final.pb"
model.save(new_model_path)

# Track end time of program
end = time.time()

# Calculate total time taken to run this program
total_time = end - start

# Print the total time taken to run this program, converted to hours and minutes
hours, remainder = divmod(total_time, 3600)
minutes, seconds = divmod(remainder, 60)
print(f"Program execution time: {int(hours)} hours, {int(minutes)} minutes, and {int(seconds)} seconds")


Epoch 1/50
363/363 [==============================] - 714s 2s/step - loss: 0.6788
Epoch 2/50
363/363 [==============================] - 710s 2s/step - loss: 0.6561
Epoch 3/50
363/363 [==============================] - 710s 2s/step - loss: 0.6318
Epoch 4/50
363/363 [==============================] - 708s 2s/step - loss: 0.6197
Epoch 5/50
363/363 [==============================] - ETA: 0s - loss: 0.6117
Epoch 5: saving model to D:\PHD_WORK\3rd_experiment\BPS_DATASET\noisy_10_model\noisy_10_G_bitplane_model\checkpoint_g_10_3\bit_denoising_model_05.pb


INFO:tensorflow:Assets written to: D:\PHD_WORK\3rd_experiment\BPS_DATASET\noisy_10_model\noisy_10_G_bitplane_model\checkpoint_g_10_3\bit_denoising_model_05.pb\assets


INFO:tensorflow:Assets written to: D:\PHD_WORK\3rd_experiment\BPS_DATASET\noisy_10_model\noisy_10_G_bitplane_model\checkpoint_g_10_3\bit_denoising_model_05.pb\assets


Epoch 5: Training Loss: 0.6117
363/363 [==============================] - 715s 2s/step - loss: 0.6117
Epoch 6/50
363/363 [==============================] - 708s 2s/step - loss: 0.6054
Epoch 7/50
363/363 [==============================] - 708s 2s/step - loss: 0.6001
Epoch 8/50
363/363 [==============================] - 710s 2s/step - loss: 0.5958
Epoch 9/50
363/363 [==============================] - 710s 2s/step - loss: 0.5913
Epoch 10/50
363/363 [==============================] - ETA: 0s - loss: 0.5863
Epoch 10: saving model to D:\PHD_WORK\3rd_experiment\BPS_DATASET\noisy_10_model\noisy_10_G_bitplane_model\checkpoint_g_10_3\bit_denoising_model_10.pb


INFO:tensorflow:Assets written to: D:\PHD_WORK\3rd_experiment\BPS_DATASET\noisy_10_model\noisy_10_G_bitplane_model\checkpoint_g_10_3\bit_denoising_model_10.pb\assets


...

INFO:tensorflow:Assets written to: D:\PHD_WORK\3rd_experiment\BPS_DATASET\noisy_10_model\noisy_10_G_bitplane_model\checkpoint_g_10_3\bit_denoising_model_50.pb\assets


Epoch 50: Training Loss: 0.4783
363/363 [==============================] - 259s 713ms/step - loss: 0.4783


INFO:tensorflow:Assets written to: D:\PHD_WORK\3rd_experiment\BPS_DATASET\bit_denoising_model_final.pb\assets


INFO:tensorflow:Assets written to: D:\PHD_WORK\3rd_experiment\BPS_DATASET\bit_denoising_model_final.pb\assets


Program execution time: 9 hours, 43 minutes, and 43 seconds
